In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
import cv2
import zipfile
import os

# Download and extract the COCO dataset
url = "http://images.cocodataset.org/zips/train2014.zip"
zip_file = tf.keras.utils.get_file("train2014.zip", url, extract=False)
with zipfile.ZipFile(zip_file, "r") as f:
    f.extractall()

# Define the model architecture
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(10, activation='softmax')
])

# Compile the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Define the image preprocessing function
def preprocess_image(image_path):
    # Load image from file
    image = cv2.imread(image_path)
    
    # Resize the image to a fixed size
    resized_image = cv2.resize(image, (224, 224))
    
    # Convert the image to RGB format
    rgb_image = cv2.cvtColor(resized_image, cv2.COLOR_BGR2RGB)
    
    # Normalize the image pixel values
    normalized_image = rgb_image / 255.0
    
    # Return the preprocessed image
    return normalized_image

# Define the data generator function
def data_generator(batch_size):
    # Load the image filenames and annotations from the COCO dataset
    image_dir = "train2014"
    annotations_file = "annotations/instances_train2014.json"
    coco = COCO(annotations_file)
    image_ids = coco.getImgIds()
    images = coco.loadImgs(image_ids)
    annotations = coco.loadAnns(coco.getAnnIds(image_ids))
    
    # Loop over the images and annotations and yield batches of data
    while True:
        batch_images = []
        batch_labels = []
        for i in range(batch_size):
            # Select a random image and its annotations
            image = np.random.choice(images)
            image_path = os.path.join(image_dir, image['file_name'])
            image_annotations = [ann for ann in annotations if ann['image_id'] == image['id']]
            
            # Preprocess the image
            preprocessed_image = preprocess_image(image_path)
            
            # Create the label tensor
            label_tensor = create_label_tensor(image_annotations)
            
            # Add the preprocessed image and label tensor to the batch
            batch_images.append(preprocessed_image)
            batch_labels.append(label_tensor)
        
        # Yield the batch of data
        yield np.array(batch_images), np.array(batch_labels)

# Define the label creation function
def create_label_tensor(annotations):
    # Create an empty label tensor
    label_tensor = np.zeros((224, 224, 10))
    
    # Loop over the annotations and fill in the label tensor
    for ann in annotations:
        category_id = ann['category_id']
        category_index = category_id - 1
        x, y, w, h = ann['bbox']
        x1 = int(x)
        y


     106496/13510573713 [..............................] - ETA: 6:24:10

KeyboardInterrupt: ignored